# Train Vehicles Agents


- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integre l'environnement pour RLlib et execute la simulation


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks.IssyOSMNetwork import IssyOSMNetwork
from flow.core.params import InFlows

## Importation du network d'Issy

On vérifie si IssyOSMNetwork est bien importé.

In [2]:
from flow.networks.IssyOSMNetwork import ADDITIONAL_NET_PARAMS, EDGES_DISTRIBUTION

print(ADDITIONAL_NET_PARAMS)
print(EDGES_DISTRIBUTION)

{'speed_limit': 50}
['-100822066', '4794817', '4783299#0', '155558218']


## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [3]:
import MinicityRouter

In [4]:
vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             num_vehicles=20)

vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             num_vehicles=5)

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

## Lance une simulation avec Training RLlib

Pour qu'un environnement puisse être entrainé, l'environnement doit être accessible via l'importation à partir de flow.envs. 


<font color='red'>
Copier l'environnement créé dans un fichier .py et on importe l'environnement dans `flow.envs.__init__.py`.
Mettre le chemin absolu du fichier .osm .
</font> 

In [5]:
import json, ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import VehicleParams, SumoCarFollowingParams

#from flow.envs import AccelEnv as myEnv
#ADDITIONAL_ENV_PARAMS = {"target_velocity": 20,"sort_vehicles": False,"max_accel": 1,"max_decel": 1}
from myenv import myEnv
ADDITIONAL_ENV_PARAMS = {"max_accel": 2, "max_decel": 2}

/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lino/anaconda3/envs/flow/lib/python3.6/site-packages

In [6]:
HORIZON = 2000

In [ ]:
# number of rollouts per training iteration
N_ROLLOUTS = 10
# number of parallel workers
N_CPUS = 2

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.1, render=False, restart_instance=True)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
path_file  = '/home/lino/Documents/DQN_CIL4SYS/DQN_CIL4SYS/notebooks/issy.osm'
net_params = NetParams(osm_path=path_file) #inflows=inflow, 

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION)


flow_params = dict( exp_tag   = "ISSY_RL_train", 
                    env_name  = myEnv,  
                    network   = IssyOSMNetwork,
                    simulator = 'traci', # simulator that is used by the experiment
                    sim       = sumo_params,
                    env       = env_params,
                    net       = net_params,
                    veh       = vehicles,
                    initial   = initial_config)

def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run   = "PPO"
    agent_cls = get_agent_class(alg_run)
    config    = agent_cls._default_config.copy()
    config["num_workers"]      = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"]            = 0.999  # discount rate
    config["use_gae"]          = True
    config["lambda"]           = 0.97
    config["kl_target"]        = 0.02
    config["num_sgd_iter"]     = 10
    config['clip_actions']     = False  # FIXME(ev) temporary ray bug
    config["horizon"]          = HORIZON
    config["model"].update({"fcnet_hiddens": [3, 3]})
    
    # save the flow params for replay
    flow_json = json.dumps( flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    
    return alg_run, gym_name, config


alg_run, gym_name, config = setup_exps()

ray.init(num_cpus=N_CPUS + 1)

exp_tag = {"run": alg_run,
           "env": gym_name,
           "config": {**config},
           "checkpoint_freq": 500,
           "checkpoint_at_end": True,
           "max_failures": 5,
           "stop": {"training_iteration": 5}}

trials = run_experiments({flow_params["exp_tag"]: exp_tag}, verbose=0)

2020-04-14 15:23:36,013	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-04-14_15-23-36_012900_9923/logs.
2020-04-14 15:23:36,126	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:63063 to respond...
2020-04-14 15:23:36,244	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:24246 to respond...
2020-04-14 15:23:36,249	INFO services.py:809 -- Starting Redis shard with 2.93 GB max memory.
2020-04-14 15:23:36,273	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-04-14_15-23-36_012900_9923/logs.
2020-04-14 15:23:36,276	WARNING services.py:1330 -- WARNING: The default object store size of 4.4 GB will use more than 50% of the available memory on this node (8.66 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2020-04-14 15:23:36,277	INFO services.py:1475 -- Starting the Plasma object store with 4.4 GB memory using

(pid=9957) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9957)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=9957) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9957)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=9957) /home/lino/anaconda3/envs/flow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9957)   _np_qint16 = np.dtype([("qint16", np.int1

(pid=9959) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.residential' (first occurence for edge '4786965#0').
(pid=9959) Warning: Discarding unknown compound 'cycleway.opposite_track' in type 'cycleway.opposite_track|highway.residential' (first occurence for edge '4786972#0').
(pid=9959) Warning: Discarding unknown compound 'cycleway.opposite' in type 'cycleway.opposite|highway.living_street' (first occurence for edge '4795758').
(pid=9959) Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.secondary' (first occurence for edge '100822066').
(pid=9959) Warning: Removed a road without junctions: -21549451
(pid=9959) Warning: Removed a road without junctions: 21549451
(pid=9959) Warning: The traffic light '30624291' does not control any links; it will not be build.
(pid=9959) Warning: Could not build program '0' for traffic light '30624291'
(pid=9959) Warning: Minor green from edge '-352962858#0' to edge '47869

(pid=9958) 2020-04-14 15:23:53,796	INFO sampler.py:304 -- Raw obs from env: { 0: { 'agent0': np.ndarray((50,), dtype=float64, min=0.0, max=1997.571, mean=490.27)}}
(pid=9958) 2020-04-14 15:23:53,796	INFO sampler.py:305 -- Info return from env: {0: {'agent0': None}}
(pid=9958) 2020-04-14 15:23:53,797	INFO sampler.py:403 -- Preprocessed obs: np.ndarray((50,), dtype=float64, min=0.0, max=1997.571, mean=490.27)
(pid=9958) 2020-04-14 15:23:53,797	INFO sampler.py:407 -- Filtered obs: np.ndarray((50,), dtype=float64, min=0.0, max=1997.571, mean=490.27)
(pid=9958) 2020-04-14 15:23:53,799	INFO sampler.py:521 -- Inputs to compute_actions():
(pid=9958) 
(pid=9958) { 'default_policy': [ { 'data': { 'agent_id': 'agent0',
(pid=9958)                                   'env_id': 0,
(pid=9958)                                   'info': None,
(pid=9958)                                   'obs': np.ndarray((50,), dtype=float64, min=0.0, max=1997.571, mean=490.27),
(pid=9958)                                 

(pid=9959) Warning: Vehicle 'human_5' performs emergency braking with decel=-24.89 wished=4.50 severity=4.53, time=117.40.
(pid=9958) Warning: Vehicle 'human_2' performs emergency braking with decel=-12.10 wished=4.50 severity=1.69, time=132.40.
(pid=9959) Warning: Vehicle 'human_13' performs emergency braking with decel=-44.34 wished=4.50 severity=8.85, time=148.30.
